# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an **investing strategy that selects the 50 stocks with the highest price momentum**. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [207]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score  # To calculate percentile scores for momentum
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [208]:
from api_secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv("sp_500_stocks.csv")

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

---

Instead of market capitalization and the price of one stock, we are going to be importing different data, these are price in one year.

In [209]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

# Looking for year1ChangePercent

[{'beta': 1.0442293975279324,
  'totalCash': 67150000000,
  'currentDebt': 123822000000,
  'revenue': 381623000000,
  'grossProfit': 173966000000,
  'totalRevenue': 381623000000,
  'EBITDA': 129629000000,
  'revenuePerShare': 24.89,
  'revenuePerEmployee': 2596074.83,
  'debtToEquity': 4.54768579669515,
  'profitMargin': 0.2630580441954494,
  'enterpriseValue': 2911506989260,
  'enterpriseValueToRevenue': 7.63,
  'priceToSales': 7.53,
  'priceToBook': 38.74,
  'forwardPERatio': 28.42097365839565,
  'pegRatio': 3.0718880160885704,
  'peHigh': 11.867282676107855,
  'peLow': 7.976472620192518,
  'week52highDate': '2023-12-14',
  'week52lowDate': '2024-04-19',
  'putCallRatio': None,
  'companyName': 'Apple Inc',
  'marketcap': 2874066989260,
  'week52high': 198.57,
  'week52low': 163.63,
  'week52highSplitAdjustOnly': 199.62,
  'week52highDateSplitAdjustOnly': '2023-12-14',
  'week52lowSplitAdjustOnly': 164.08,
  'week52lowDateSplitAdjustOnly': '2024-04-19',
  'week52change': 0.1008645160

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [210]:
data[0]["year1ChangePercent"]    #Increase of 9.094018% of benefit.

0.09767510503549603

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [211]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [212]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for position in range(0,len(symbol_string.split(','))):
        final_dataframe.loc[len(final_dataframe.index)]=[data[position]["symbol"], data[position]["latestPrice"], data[position + int(len(data)/2)]["year1ChangePercent"], "N/A"]
    
final_dataframe

C:\Users\Manu\AppData\Local\Temp\ipykernel_3116\1732865893.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe.loc[len(final_dataframe.index)]=[data[position]["symbol"], data[position]["latestPrice"], data[position + int(len(data)/2)]["year1ChangePercent"], "N/A"]
C:\Users\Manu\AppData\Local\Temp\ipykernel_3116\1732865893.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe.loc[len(final_dataframe.index)]=[data[position]["symbol"], data[position]["latestPrice"], da

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,151.2800,0.195219,N/A
1,AAL,15.1900,0.083452,N/A
2,AAP,75.6400,-0.374567,N/A
3,AAPL,187.4300,0.097675,N/A
4,ABBV,161.5900,0.142706,N/A
...,...,...,...,...
500,YUM,137.4900,0.011717,N/A
501,ZBH,119.9700,-0.115923,N/A
502,ZBRA,320.9800,0.205966,N/A
503,ZION,44.1357,1.091186,N/A


In [213]:
"""
symbol_prueba = "AAPL"
batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol_prueba}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json() 
for position in len(symbol_string.split(',')):
    print(position)
"""
"""
for symbol_string in symbol_strings:
    print(symbol_string)
    for position in range(0,len(symbol_string.split(','))):
        print(position)
"""

#data[0]["symbol"], data[0]["latestPrice"],data[1]["year1ChangePercent"]

"\nfor symbol_string in symbol_strings:\n    print(symbol_string)\n    for position in range(0,len(symbol_string.split(','))):\n        print(position)\n"

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [214]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)

#Now it takes only the first 50 rows
final_dataframe = final_dataframe[:50].reset_index()
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.9200,2.281287,N/A
1,345,NVDA,913.5600,2.224757,N/A
2,208,GPS,21.5200,1.848219,N/A
3,413,SLG,54.7700,1.774696,N/A
4,340,NRG,82.0800,1.584944,N/A
5,37,ANET,313.6600,1.262569,N/A
6,50,AVGO,1380.0300,1.261546,N/A
7,479,WDC,73.5400,1.226461,N/A
8,462,URI,694.4700,1.117664,N/A
9,503,ZION,44.1357,1.091186,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [215]:
"""
money_for_investing = input("How much money do you want to invest?")

try:
    val = float(money_for_investing)
except ValueError:
    print("The value you have introduced is not a real value. Try again.")
    input("How much money do you want to invest?")
    val = float(money_for_investing)
"""

'\nmoney_for_investing = input("How much money do you want to invest?")\n\ntry:\n    val = float(money_for_investing)\nexcept ValueError:\n    print("The value you have introduced is not a real value. Try again.")\n    input("How much money do you want to invest?")\n    val = float(money_for_investing)\n'

In [216]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("How much money do you want to invest?")
    try:
        val = float(portfolio_size)
    except ValueError:
        print("The value you have introduced is not a real value. Try again.")
        portfolio_size = input("How much money do you want to invest?")
        val = float(portfolio_size)

portfolio_input()


In [217]:
position_size = float(portfolio_size)/len(final_dataframe.index)
position_size  #The part for each of the 50 stocks, remember that we have modified the quantity of stocks!

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i,"Price"])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.9200,2.281287,250
1,345,NVDA,913.5600,2.224757,21
2,208,GPS,21.5200,1.848219,929
3,413,SLG,54.7700,1.774696,365
4,340,NRG,82.0800,1.584944,243
5,37,ANET,313.6600,1.262569,63
6,50,AVGO,1380.0300,1.261546,14
7,479,WDC,73.5400,1.226461,271
8,462,URI,694.4700,1.117664,28
9,503,ZION,44.1357,1.091186,453


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [218]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for position in range(0,len(symbol_string.split(','))):
        hqm_dataframe.loc[len(hqm_dataframe.index)]=[
            data[position]["symbol"], 
            data[position]["latestPrice"], 
            "N/A",
            data[position + int(len(data)/2)]["year1ChangePercent"],
            "N/A",
            data[position + int(len(data)/2)]["month6ChangePercent"],
            "N/A",
            data[position + int(len(data)/2)]["month3ChangePercent"],
            "N/A",
            data[position + int(len(data)/2)]["month1ChangePercent"],
            "N/A"
            ]
   
hqm_dataframe 

C:\Users\Manu\AppData\Local\Temp\ipykernel_3116\4076160398.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe.loc[len(hqm_dataframe.index)]=[
C:\Users\Manu\AppData\Local\Temp\ipykernel_3116\4076160398.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe.loc[len(hqm_dataframe.index)]=[
C:\Users\Manu\AppData\Local\Temp\ipykernel_3116\4076160398.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,151.2800,N/A,0.195219,N/A,0.359933,N/A,0.145839,N/A,0.074966,N/A
1,AAL,15.1900,N/A,0.083452,N/A,0.240000,N/A,0.022207,N/A,0.155133,N/A
2,AAP,75.6400,N/A,-0.374567,N/A,0.304723,N/A,0.180880,N/A,0.072908,N/A
3,AAPL,187.4300,N/A,0.097675,N/A,0.005221,N/A,0.020574,N/A,0.064506,N/A
4,ABBV,161.5900,N/A,0.142706,N/A,0.192651,N/A,-0.064906,N/A,-0.004252,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.4900,N/A,0.011717,N/A,0.088619,N/A,0.031103,N/A,-0.002250,N/A
501,ZBH,119.9700,N/A,-0.115923,N/A,0.114420,N/A,-0.013496,N/A,-0.036231,N/A
502,ZBRA,320.9800,N/A,0.205966,N/A,0.488361,N/A,0.268896,N/A,0.108050,N/A
503,ZION,44.1357,N/A,1.091186,N/A,0.272806,N/A,0.102177,N/A,0.095402,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [219]:
time_periods = [
        'One-Year',
        'Six-Month',
        'Three-Month',
        'One-Month'
    ]

#Null value control

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(                   # stats.percentileofscore
            hqm_dataframe[change_col], 
            hqm_dataframe.loc[row,change_col],
            nan_policy='omit')/100                                        # Null value control
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,151.2800,N/A,0.195219,0.531828,0.359933,0.87885,0.145839,0.772074,0.074966,0.741273
1,AAL,15.1900,N/A,0.083452,0.351129,0.240000,0.712526,0.022207,0.392197,0.155133,0.946612
2,AAP,75.6400,N/A,-0.374567,0.016427,0.304723,0.813142,0.180880,0.868583,0.072908,0.722793
3,AAPL,187.4300,N/A,0.097675,0.38809,0.005221,0.164271,0.020574,0.38193,0.064506,0.683778
4,ABBV,161.5900,N/A,0.142706,0.464066,0.192651,0.632444,-0.064906,0.160164,-0.004252,0.316222
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.4900,N/A,0.011717,0.26078,0.088619,0.355236,0.031103,0.425051,-0.002250,0.326489
501,ZBH,119.9700,N/A,-0.115923,0.123203,0.114420,0.420945,-0.013496,0.283368,-0.036231,0.195072
502,ZBRA,320.9800,N/A,0.205966,0.556468,0.488361,0.965092,0.268896,0.963039,0.108050,0.874743
503,ZION,44.1357,N/A,1.091186,0.98152,0.272806,0.7577,0.102177,0.644764,0.095402,0.831622


In [220]:
for time_period in time_periods:
    print(time_period)

One-Year
Six-Month
Three-Month
One-Month


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [221]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles=[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.2800,N/A,0.195219,0.531828,0.359933,0.87885,0.145839,0.772074,0.074966,0.741273,0.731006
1,AAL,15.1900,N/A,0.083452,0.351129,0.240000,0.712526,0.022207,0.392197,0.155133,0.946612,0.600616
2,AAP,75.6400,N/A,-0.374567,0.016427,0.304723,0.813142,0.180880,0.868583,0.072908,0.722793,0.605236
3,AAPL,187.4300,N/A,0.097675,0.38809,0.005221,0.164271,0.020574,0.38193,0.064506,0.683778,0.404517
4,ABBV,161.5900,N/A,0.142706,0.464066,0.192651,0.632444,-0.064906,0.160164,-0.004252,0.316222,0.393224
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.4900,N/A,0.011717,0.26078,0.088619,0.355236,0.031103,0.425051,-0.002250,0.326489,0.341889
501,ZBH,119.9700,N/A,-0.115923,0.123203,0.114420,0.420945,-0.013496,0.283368,-0.036231,0.195072,0.255647
502,ZBRA,320.9800,N/A,0.205966,0.556468,0.488361,0.965092,0.268896,0.963039,0.108050,0.874743,0.839836
503,ZION,44.1357,N/A,1.091186,0.98152,0.272806,0.7577,0.102177,0.644764,0.095402,0.831622,0.803901


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [222]:
hqm_dataframe.sort_values("HQM Score", ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True,drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HWM,80.8800,N/A,0.845990,0.956879,0.572476,0.98152,0.276896,0.967146,0.264526,0.99384,0.974846
1,NRG,82.0800,N/A,1.584944,0.991786,0.757440,0.995893,0.585379,1.0,0.116391,0.897331,0.971253
2,GRMN,169.9100,N/A,0.677953,0.911704,0.444028,0.938398,0.390134,0.991786,0.187269,0.979466,0.955339
3,DAL,53.0600,N/A,0.611463,0.89117,0.512133,0.973306,0.304788,0.975359,0.134466,0.930185,0.942505
4,ANET,313.6600,N/A,1.262569,0.989733,0.457528,0.948665,0.180638,0.86653,0.156478,0.950719,0.938912
5,APH,128.5600,N/A,0.727767,0.934292,0.455364,0.944559,0.237339,0.942505,0.134486,0.932238,0.938398
6,LDOS,146.6000,N/A,0.885004,0.963039,0.405460,0.911704,0.210175,0.911704,0.165991,0.963039,0.937372
7,QCOM,188.9700,N/A,0.918651,0.967146,0.517983,0.975359,0.240059,0.948665,0.103217,0.858316,0.937372
8,WFC,61.8900,N/A,0.700093,0.924025,0.489617,0.967146,0.282625,0.969199,0.102281,0.850103,0.927618
9,WAB,165.1500,N/A,0.695996,0.919918,0.431479,0.928131,0.224318,0.930185,0.129772,0.928131,0.926591


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [223]:
portfolio_input()

In [224]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[i,"Price"])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HWM,80.8800,247,0.845990,0.956879,0.572476,0.98152,0.276896,0.967146,0.264526,0.99384,0.974846
1,NRG,82.0800,243,1.584944,0.991786,0.757440,0.995893,0.585379,1.0,0.116391,0.897331,0.971253
2,GRMN,169.9100,117,0.677953,0.911704,0.444028,0.938398,0.390134,0.991786,0.187269,0.979466,0.955339
3,DAL,53.0600,376,0.611463,0.89117,0.512133,0.973306,0.304788,0.975359,0.134466,0.930185,0.942505
4,ANET,313.6600,63,1.262569,0.989733,0.457528,0.948665,0.180638,0.86653,0.156478,0.950719,0.938912
5,APH,128.5600,155,0.727767,0.934292,0.455364,0.944559,0.237339,0.942505,0.134486,0.932238,0.938398
6,LDOS,146.6000,136,0.885004,0.963039,0.405460,0.911704,0.210175,0.911704,0.165991,0.963039,0.937372
7,QCOM,188.9700,105,0.918651,0.967146,0.517983,0.975359,0.240059,0.948665,0.103217,0.858316,0.937372
8,WFC,61.8900,323,0.700093,0.924025,0.489617,0.967146,0.282625,0.969199,0.102281,0.850103,0.927618
9,WAB,165.1500,121,0.695996,0.919918,0.431479,0.928131,0.224318,0.930185,0.129772,0.928131,0.926591


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [225]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name="Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [226]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [227]:
column_formats = {
    'A': ['Ticker',string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return',percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets["Momentum Strategy"].set_column(f'{column}:{column}',25,column_formats[column][1])
    writer.sheets["Momentum Strategy"].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [228]:
writer.close()